In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, wordpunct_tokenize
import nltk

In [2]:
data = pd.read_csv("0000000000002747_training_twitter_x_y_train.csv")

In [3]:
test = pd.read_csv("0000000000002747_test_twitter_x_test.csv")

In [4]:
data.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [5]:
set(data.airline)

{'American', 'Delta', 'Southwest', 'US Airways', 'United', 'Virgin America'}

In [6]:
text = data.text
test_text = test.text

In [7]:
y = data.airline_sentiment
def categorize(s):
    if s == 'negative':
        return 1
    elif s == 'neutral':
        return 2
    else:
        return 3
y = y.apply(categorize)

In [8]:
print((y == 1).sum())
print((y == 2).sum())
print((y == 3).sum())

6851
2327
1802


In [9]:
stop = stopwords.words('english')

In [10]:
stop.remove('not')
import string
punc = list(string.punctuation)
stop += punc

In [11]:
def tokenize(txt):
    return wordpunct_tokenize(txt.lower())
text = text.apply(tokenize)
test_text = test_text.apply(tokenize)

In [12]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

In [13]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [14]:
lemmatizer = WordNetLemmatizer()

In [15]:
def clean(words):
    return [lemmatizer.lemmatize(w, pos=get_simple_pos(pos_tag([w])[0][1])) for w in words if not w in stop]
clean_text = text.apply(clean)
test_text = test_text.apply(clean)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
x = [" ".join(arr) for arr in clean_text.values]
x_pred = [" ".join(arr) for arr in test_text.values]

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y.values)

In [19]:
count_vec = CountVectorizer(max_features = 170)
a = count_vec.fit_transform(x_train)

In [20]:
print(len(count_vec.get_feature_names()))

170


In [21]:
x_train_1 = a.todense()
x_test_1 = count_vec.transform(x_test).todense()

In [22]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
svc = SVC(C=1e3,gamma=1e-3,random_state=0)

In [23]:
svc.fit(x_train_1, y_train)

SVC(C=1000.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [24]:
svc.score(x_test_1, y_test)

0.75591985428051001

In [25]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(criterion="entropy", min_impurity_decrease=10**-8, n_estimators=1000, max_depth=25, n_jobs=-1, random_state=0)
rf.fit(x_train_1, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=1e-08, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [26]:
rf.score(x_test_1, y_test)

0.73224043715846998

In [27]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_jobs=-1, n_neighbors=25, weights='distance')
knn.fit(x_train_1, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=25, p=2,
           weights='distance')

In [28]:
knn.score(x_test_1,y_test)

0.6262295081967213

In [29]:
from sklearn.naive_bayes import MultinomialNB

In [30]:
mnb = MultinomialNB(alpha=0.1)

In [31]:
mnb.fit(x_train_1, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [32]:
mnb.score(x_test_1, y_test)

0.73406193078324222